# LA Fire — Concise Validation Notebook

This notebook provides a compact, reproducible preview of the 600 m grid and which cells were kept vs removed after pruning. It also includes an optional small-panel generator for kept cells.

In [1]:
# Paths and imports
from pathlib import Path
import geopandas as gpd
import fiona, csv, re
import folium

BASE = Path("/media/gisense/xihan/250812_CyberTraining_Team4")
GPKG = BASE / 'data' / 'interim' / 'la_fire_grid_600m_clip_utm11.gpkg'
CHIPS = BASE / 'data' / 'chips_600m'
BACKUPS = BASE / 'backups'
OUT_HTML = BASE / 'data' / 'interim' / 'la_fire_grid_600m_clip_preview.html'
PANELS = BASE / 'data' / 'derived' / 'pre_post_panels'
PANELS.mkdir(parents=True, exist_ok=True)

# 1) Current cells (those with a folder in data/chips_600m)
if CHIPS.exists():
    current_cells = sorted([p.name for p in CHIPS.iterdir() if p.is_dir()])
else:
    current_cells = []

# 2) Load removed_cells.csv from latest backup if available
removed_cells = set()
if BACKUPS.exists():
    for b in sorted([p for p in BACKUPS.iterdir() if p.is_dir()], reverse=True):
        rcsv = b / 'removed_cells.csv'
        if rcsv.exists():
            try:
                with rcsv.open() as f:
                    r = csv.DictReader(f)
                    for row in r:
                        if 'cell_name' in row and row['cell_name']:
                            removed_cells.add(row['cell_name'])
                break
            except Exception:
                continue

# 3) Fallback: infer removed set from backup manifest if list empty
if not removed_cells and BACKUPS.exists():
    for b in sorted([p for p in BACKUPS.iterdir() if p.is_dir()], reverse=True):
        man = b / 'chips_600m_manifest.csv'
        if man.exists():
            orig = set()
            with man.open() as f:
                rdr = csv.DictReader(f)
                for row in rdr:
                    if 'cell_id' in row and row['cell_id']:
                        try:
                            cid = int(row['cell_id'])
                            orig.add(f'cell_{cid:05d}')
                            continue
                        except Exception:
                            pass
                    for v in row.values():
                        if not v:
                            continue
                        m = re.search(r'(cell_\d{5})', str(v))
                        if m:
                            orig.add(m.group(1))
                            break
            removed_cells = orig - set(current_cells)
            break

# 4) Final sets
kept_set = set(current_cells)
removed_set = set(removed_cells)
print(f'Kept cells: {len(kept_set)} | Removed cells (from backup): {len(removed_set)}')

# 5) Load grid geometries and prepare GeoDataFrames for mapping
if not GPKG.exists():
    raise FileNotFoundError(f'Grid geopackage not found: {GPKG}')
layers = fiona.listlayers(str(GPKG))
layer = layers[0] if layers else None
grid = gpd.read_file(GPKG, layer=layer)
if 'cell_id' not in grid.columns:
    grid = grid.reset_index().rename(columns={'index':'cell_id'})

def _fmt_cell_name(v):
    try:
        return f'cell_{int(v):05d}'
    except Exception:
        s = str(v)
        m = re.search(r'(cell_\d{5})', s)
        if m:
            return m.group(1)
        return s

grid['cell_name'] = grid['cell_id'].apply(_fmt_cell_name)
grid_w = grid.to_crs(4326)
kept_w = grid_w[grid_w['cell_name'].isin(kept_set)].copy()
removed_w = grid_w[grid_w['cell_name'].isin(removed_set)].copy()

# 6) Render interactive map and save (kept green, removed red dashed)
m = kept_w.explore(color='#2ca25f', style_kwds={'fillOpacity':0.15, 'weight':0.6}, name='Kept cells')
if not removed_w.empty:
    removed_w.explore(m=m, color='#de2d26', style_kwds={'fillOpacity':0, 'weight':1.0, 'dashArray':'6,6'}, name='Removed cells')
folium.LayerControl().add_to(m)
m.save(OUT_HTML.as_posix())
print('Saved preview HTML →', OUT_HTML)


Kept cells: 295 | Removed cells (from backup): 248
Saved preview HTML → /media/gisense/xihan/250812_CyberTraining_Team4/data/interim/la_fire_grid_600m_clip_preview.html
Saved preview HTML → /media/gisense/xihan/250812_CyberTraining_Team4/data/interim/la_fire_grid_600m_clip_preview.html


In [ ]:
# Optional: generate small PRE+POST panels for kept cells (fast; limit with MAX_CELLS)
from pathlib import Path
import rasterio, numpy as np, matplotlib.pyplot as plt

MAX_CELLS = 50  # set to None to process all; keep small to avoid long runs
POSTS_PER_CELL = 3

def read_rgb(tif_path):
    try:
        with rasterio.open(tif_path) as ds:
            bidx = [1,2,3] if ds.count >= 3 else list(range(1, ds.count+1))
            arr = ds.read(bidx).astype('float32')
            if arr.max() > 1.5:
                arr = np.clip(arr / 255.0, 0, 1)
            arr = np.transpose(arr, (1,2,0))
            return arr
    except Exception:
        return None

kept_list = sorted(kept_set)
to_do = kept_list if MAX_CELLS is None else kept_list[:int(MAX_CELLS)]
print(f'Generating panels for {len(to_do)} cells (limit {MAX_CELLS})')

for i, cname in enumerate(to_do, start=1):
    cell_dir = CHIPS / cname
    pre_files = sorted((cell_dir / 'pre').glob('*.tif')) if (cell_dir / 'pre').exists() else []
    post_files = sorted((cell_dir / 'post').rglob('*.tif')) if (cell_dir / 'post').exists() else []
    if not pre_files or not post_files:
        continue
    pre = read_rgb(pre_files[0])
    posts = [read_rgb(p) for p in post_files[:POSTS_PER_CELL]]
    # build simple horizontal panel: pre then posts
,
,
4
,
1